In [1]:
%%writefile matmul.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Kernel 定义
__global__ void matrix_multiplication_kernel(const float* A, const float* B, float* C, int M, int N, int K) {
    int row =  blockDim.y * blockIdx.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if(col >= K || row >= M){
        return;
    }
    
    float acc = 0.0f;
    for(int i = 0; i < N; i++){
        acc += A[row * N + i] * B[i * K + col];
    }
    C[row * K + col] = acc; 
}

// 宿主端 wrapper 函数
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 threadsPerBlock(16, 16);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrix_multiplication_kernel<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);
    
    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }
    
    // 等待 GPU 完成
    cudaDeviceSynchronize();
}

Overwriting matmul.cu


上面的实现有一个很大的问题就是，在每一个线程里面，为了计算C里面的一个位置上面的值，需要去全局内存上访问col次数据A，也需要访问col次数据B，导致数据等待时间大大增加。解决方法在于，使用线程块里面的共享内存，在一个线程块里面，先读取A和B的一部分数据到线程块里面的共享内存里面，然后进行计算，接着读取下一部分。通过分块避免所有线程去读取全局内存里面的数据。

In [2]:
%%writefile matmul_v2.cu
#include <stdio.h>
#include <cuda_runtime.h>
#define TILE_WIDTH 32 // 假设 BlockDim 为 32x32

__global__ void matrix_multiplication_shared_mem(const float* __restrict__ A, const float* __restrict__ B, float* C, int M, int N, int K) {
    // 申请共享内存
    __shared__ float As[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // 计算当前线程在全局矩阵 C 中负责的坐标
    int row = by * TILE_WIDTH + ty;
    int col = bx * TILE_WIDTH + tx;

    float acc = 0.0f;

    // 循环遍历所有的 Tile (以 TILE_WIDTH 为步长遍历 N 维度)
    for (int t = 0; t < (N + TILE_WIDTH - 1) / TILE_WIDTH; ++t) {

        // 1. 协作加载 A 的 Tile 到共享内存
        // 边界检查：防止索引越界 (Padding 0)
        if (row < M && t * TILE_WIDTH + tx < N) {
            As[ty][tx] = A[row * N + t * TILE_WIDTH + tx];
        } else {
            As[ty][tx] = 0.0f;
        }

        // 2. 协作加载 B 的 Tile 到共享内存
        if (col < K && t * TILE_WIDTH + ty < N) {
            // 注意这里 B 的索引：行是 t*TILE_WIDTH + ty, 列是 col
            Bs[ty][tx] = B[(t * TILE_WIDTH + ty) * K + col];
        } else {
            Bs[ty][tx] = 0.0f;
        }

        // 3. 必须同步！确保所有线程都加载完了数据
        __syncthreads();

        // 4. 在共享内存上进行计算
        for (int i = 0; i < TILE_WIDTH; ++i) {
            acc += As[ty][i] * Bs[i][tx];
        }

        // 5. 必须同步！确保在加载下一个 Tile 之前，当前 Tile 的数据已经用完
        __syncthreads();
    }

    // 写回结果
    if (row < M && col < K) {
        C[row * K + col] = acc;
    }
}

// 宿主端 wrapper 函数
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 threadsPerBlock(TILE_WIDTH, TILE_WIDTH);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrix_multiplication_shared_mem<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);
    
    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }
    
    // 等待 GPU 完成
    cudaDeviceSynchronize();
}

Overwriting matmul_v2.cu


In [3]:
%%writefile matmul_v3.cu
#include <stdio.h>
#include <cuda_runtime.h>
#define TILE_WIDTH 32 // 假设 BlockDim 为 32x32

__global__ void matrix_multiplication_shared_mem(const float *__restrict__ A, const float *__restrict__ B, float *C, int M, int N, int K)
{
    // 申请共享内存
    __shared__ float As[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];

    int ty = threadIdx.y;
    int tx = threadIdx.x;

    int row = blockIdx.y * TILE_WIDTH + ty;
    int col = blockIdx.x * TILE_WIDTH + tx;

    float acc = 0.0f;

    for (int i = 0; i < (N + TILE_WIDTH - 1) / TILE_WIDTH; i++)
    {
        // 读取数据
        if (row < M && TILE_WIDTH * i + tx < N)
        {
            As[ty][tx] = A[row * N + TILE_WIDTH * i + tx];
        }
        else
        {
            As[ty][tx] = 0.0f;
        }
        if (col < K && TILE_WIDTH * i + ty < N)
        {
            Bs[tx][ty] = B[(TILE_WIDTH * i + ty) * K + col];
        }
        else
        {
            Bs[tx][ty] = 0.0f;
        }
        __syncthreads(); // 等待线程块里面的线程都搬运完成

        for (int j = 0; j < TILE_WIDTH; j++)
        {
            acc += As[ty][j] * Bs[tx][j]; //访问线程块里面的共享内存时，没有合并访问，只在乎bank config
        }

        __syncthreads();
    }
    if (row<M && col<K)
    {
        C[row*K+col] = acc;
    }
}

// 宿主端 wrapper 函数
extern "C" void solve(const float *A, const float *B, float *C, int M, int N, int K)
{
    dim3 threadsPerBlock(TILE_WIDTH, TILE_WIDTH);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);

    matrix_multiplication_shared_mem<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);

    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess)
    {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }

    // 等待 GPU 完成
    cudaDeviceSynchronize();
}


Overwriting matmul_v3.cu


In [4]:
import os
import subprocess
for v in os.listdir(os.path.abspath('.')):
    prefix, end = os.path.splitext(v)
    if end == '.cu':
        subprocess.run(f"nvcc -shared -o {prefix}.so {prefix}.cu -Xcompiler -fPIC", shell=True)
print(os.listdir())

['matmul_v2.cu', 'matmul_v2.so', '.virtual_documents', 'matmul_v3.cu', 'matmul_v3.so', 'matmul.cu', 'matmul.so']


In [5]:
import torch
import ctypes
import numpy as np

flag = True
for v in os.listdir('.'):
    prefix, end = os.path.splitext(v)
    
    if end != '.so':
        continue

    # 1. 加载编译好的 .so 库
    lib = ctypes.CDLL(f'./{v}')

    # 2. 定义函数参数类型
    # void solve(const float* A, const float* B, float* C, int M, int N, int K)
    # 指针对应 c_void_p (因为我们要传显存地址), int 对应 c_int
    lib.solve.argtypes = [
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_int, 
        ctypes.c_int, 
        ctypes.c_int
    ]

    def cuda_matmul(a_tensor, b_tensor):
        # 获取维度
        M, N = a_tensor.shape
        N_b, K = b_tensor.shape
        
        assert N == N_b, f"矩阵维度不匹配: {N} != {N_b}"
        
        # 初始化输出矩阵 C (在 GPU 上分配)
        c_tensor = torch.zeros((M, K), device='cuda', dtype=torch.float32)
        
        # 确保输入是连续内存且在 GPU 上
        if not a_tensor.is_contiguous(): a_tensor = a_tensor.contiguous()
        if not b_tensor.is_contiguous(): b_tensor = b_tensor.contiguous()
        
        # 3. 调用 CUDA 函数
        # 注意：必须传入 data_ptr()，这是物理显存地址
        lib.solve(
            ctypes.c_void_p(a_tensor.data_ptr()),
            ctypes.c_void_p(b_tensor.data_ptr()),
            ctypes.c_void_p(c_tensor.data_ptr()),
            ctypes.c_int(M),
            ctypes.c_int(N),
            ctypes.c_int(K)
        )
        
        return c_tensor

    # --- 测试部分 ---

    # 设置维度
    M, N, K = 1024, 512, 1024

    print(f"正在进行矩阵乘法测试: [{M}x{N}] * [{N}x{K}]")

    # 创建随机数据 (在 GPU 上)
    A = torch.randn(M, N, device='cuda', dtype=torch.float32)
    B = torch.randn(N, K, device='cuda', dtype=torch.float32)

    # 1. 运行你的 CUDA Kernel
    C_custom = cuda_matmul(A, B)

    # 2. 运行 PyTorch 内置矩阵乘法 (作为标准答案)
    C_torch = torch.matmul(A, B)
    
    if flag:
        print("Pytorch: ")
        %timeit torch.matmul(A, B); torch.cuda.synchronize()
        flag = False
        print()
    print(f"{v}")
    # 3. 验证结果
    # 允许一点浮点误差
    if torch.allclose(C_custom, C_torch, atol=1e-3):
        print(f"✅ 测试通过！结果正确。")
        %timeit cuda_matmul(A, B); torch.cuda.synchronize()
    else:
        print("❌ 测试失败。结果不一致。")
        print("最大误差:", (C_custom - C_torch).abs().max().item())
    print()

正在进行矩阵乘法测试: [1024x512] * [512x1024]
Pytorch: 
348 µs ± 2.54 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

matmul_v2.so
✅ 测试通过！结果正确。
1.23 ms ± 6.18 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

正在进行矩阵乘法测试: [1024x512] * [512x1024]
matmul_v3.so
✅ 测试通过！结果正确。
5.19 ms ± 6.15 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

正在进行矩阵乘法测试: [1024x512] * [512x1024]
matmul.so
✅ 测试通过！结果正确。
2.23 ms ± 6.52 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

